In [ ]:
def detect_marker (img, color):
    #color = [hsv_min, hsv_max]
    
    # преобразуем RGB картинку в HSV модель
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # применяем цветовой фильтр
    thresh = cv2.inRange(hsv, color[0], color[1])
    
    # вычисляем моменты изображения
    moments = cv2.moments(thresh, 1)
    dM01 = moments['m01']
    dM10 = moments['m10']
    dArea = moments['m00']
    # будем реагировать только на те моменты,
    # которые содержать больше 100 пикселей
    
    marker_coord = [0, 0]
    if dArea > 100:
        x = int(dM10 / dArea)
        y = int(dM01 / dArea)
        marker_coord[0] = x
        marker_coord[1] = y
        #cv2.circle(img, (x, y), 10, (0,0,255), -1)
    return marker_coord

In [ ]:
def find_marker_place (img, marker_coord, size_cell_x, size_cell_y, color):
     coord_rectang = [0, 0, 0, 0]
        k_i = 0
        l_j = 0
        while k_i <= img.shape[1]:
            while l_j <= img.shape[0]:
                if marker_coord[0] < k_i + size_cell_x and marker_coord[0] > k_i and marker_coord[1] < l_j + size_cell_y and marker_coord[1] > l_j:
                    coord_rectang[0] = k_i
                    coord_rectang[1] = l_j
                    coord_rectang[2] = k_i+size_cell_x
                    coord_rectang[3] = l_j+size_cell_y
                    #cv2.rectangle(img,(k_i, l_j),(k_i+dev1, l_j+dev2),(0,255,255),3)                  
                l_j+= dev2
            k_i+= dev1
            l_j = 0

        return coord_rectang

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    
    lst = list(dim)
    lst[0] = int(lst[0])
    dim = tuple(lst)
    
    resized = cv2.resize(image, dim, interpolation = inter)
    
    # return the resized image
    return resized


def find_movement_of_man (curr_frame, back_frame, connectivity = 4):
    
    #shape
    shape = curr_frame.shape
    
    #make a photo of curr_frame
    new_frame = cv2.imwrite('opencv1.png', curr_frame)
    
    #make a resize
    back_frame_resize = image_resize(back_frame , width = shape[1]/4, height = shape[0]/4)
    new_frame_resize = image_resize(new_frame , width = shape[1]/4, height = shape[0]/4)
    
    #filter
    filtered_back_frame = cv2.blur (back_frame, (9, 9))
    filtered_new_frame = cv2.blur (new_frame, (9, 9))
    
    #difference between photos
    diff_between = cv2.absdiff(filtered_back_frame,filtered_new_frame)
    

    img_grey = cv2.cvtColor(diff_between, cv2.COLOR_BGR2GRAY)
    
    gaussian_blur = cv2.GaussianBlur(img_grey, (5,5), 0)
    
    #binarization
    _, frame = cv2.threshold(gaussian_blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    kernel = np.ones((10,10),np.uint8)
    erosia = cv2.erode(frame,kernel,iterations = 1)
    
    frame_end = image_resize(erosia , width = sh[1], height = sh[0])
    
    #find the lagest component of connectivity
    output = cv2.connectedComponentsWithStats(d, connectivity, cv2.CV_32S)
    
    #rectangle around area
    
    new_rectangle_coord = [0, 0, 0, 0]
    for i in range(output[0]):
            if output[2][i][4] >= 5000: 
                new_rectangle_coord[0] = output[2][i][0]
                new_rectangle_coord[1] = output[2][i][1]
                new_rectangle_coord[2] = output[2][i][0] + output[2][i][2]
                new_rectangle_coord[3] = output[2][i][1] + output[2][i][3]
                #cv2.rectangle(d, (output[2][i][0], output[2][i][1]), 
                ####(output[2][i][0] + output[2][i][2], output[2][i][1] + output[2][i][3]), (255,0,255), 2)
    
    return new_rectangle_coord

In [ ]:
class marker_tracker:
    
    up = [0, 0]
    right = [0, 0]
    left = [0, 0]
    down = [0, 0]
    
    size_cell_x = 0
    size_cell_y = 0
    
    x_array = []
    y_array = []
    
    width = 0
    height = 0
    
    
    def __init__(self, cam_num, cellx, celly, colors):
        self.cam_num = cam_num
        self.cellx = cellx
        self.celly = celly
        self.colors = colors
        
        cam = cv2.VideoCapture(cam_num)
        flag, img = cam.read()
        shape = img.shape
        
        width = shape[0]
        height = shape[1]
        
        size_cell_x = width//cellx
        size_cell_y = height//celly
        
        
    def draw_grid(self):
        i = 0
        j = 0   
        while i <= width:#img.shape[1]:#width:
            while j <= height:#img.shape[0]:#height:
                cv2.rectangle(img,(i, j),(i+size_cell_x, j+size_cell_y),(255,255,255),1)
                j+= size_cell_y
            i+= size_cell_x
            j = 0
    
    
#     def measurments(x_array, y_array):
#         ###find top position###
#         top = [0, 0]
#         for i in range(len(y_array)):
#             if y_array[i] > top[1]:
#                 top[0] = x_array[i]
#                 top[1] = y_array[i]
#         ###find left position###
#         left = [0, 0]
#         left[0] = 10000
#         for i in range(len(x_array)):
#             if x_array[i] < left[0]:
#                 left[0] = x_array[i]
#                 left[1] = y_array[i]
#         ###find right position###
#         right = [0, 0]
#         right[0] = 0
#         for i in range(len(x_array)):
#             if x_array[i] > right[0]:
#                 right[0] = x_array[i]
#                 right[1] = y_array[i]
#         ###find botom position###
#         botom = [0, 0]
#         botom[1] = 10000
#         for i in range(len(y_array)):
#             if y_array[i] < botom[1]:
#                 botom[0] = x_array[i]
#                 botom[1] = y_array[i]

#         return top, left, right, botom

    def calibration(self, finish):
        if finish == True:
            find()
            x_array.append(x_marker)
            y_array.append(y_marker)
        else:
            ###find top position###
            top = [0, 0]
            for i in range(len(y_array)):
                if y_array[i] > top[1]:
                    top[0] = x_array[i]
                    top[1] = y_array[i]
            ###find left position###
            left = [0, 0]
            left[0] = 10e10
            for i in range(len(x_array)):
                if x_array[i] < left[0]:
                    left[0] = x_array[i]
                    left[1] = y_array[i]
            ###find right position###
            right = [0, 0]
            right[0] = 0
            for i in range(len(x_array)):
                if x_array[i] > right[0]:
                    right[0] = x_array[i]
                    right[1] = y_array[i]
            ###find botom position###
            bottom = [0, 0]
            bottom[1] = 10e10
            for i in range(len(y_array)):
                if y_array[i] < botom[1]:
                    bottom[0] = x_array[i]
                    bottom[1] = y_array[i]
                    
        return top, left, right, bottom



    def detect_marker (self, img, color):
        #color = [hsv_min, hsv_max]

        # преобразуем RGB картинку в HSV модель
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        # применяем цветовой фильтр
        thresh = cv2.inRange(hsv, color[0], color[1])

        # вычисляем моменты изображения
        moments = cv2.moments(thresh, 1)
        dM01 = moments['m01']
        dM10 = moments['m10']
        dArea = moments['m00']
        # будем реагировать только на те моменты,
        # которые содержать больше 100 пикселей

        marker_coord = [0, 0]
        if dArea > 100:
            x = int(dM10 / dArea)
            y = int(dM01 / dArea)
            marker_coord[0] = x
            marker_coord[1] = y
            #cv2.circle(img, (x, y), 10, (0,0,255), -1)
        return marker_coord



    def find_marker_place (self, img, marker_coord, size_cell_x, size_cell_y, color):
        coord_rectang = [0, 0, 0, 0]
        k_i = 0
        l_j = 0
        while k_i <= img.shape[1]:
            while l_j <= img.shape[0]:
                if marker_coord[0] < k_i + size_cell_x and marker_coord[0] > k_i and marker_coord[1] < l_j + size_cell_y and marker_coord[1] > l_j:
                    coord_rectang[0] = k_i
                    coord_rectang[1] = l_j
                    coord_rectang[2] = k_i+size_cell_x
                    coord_rectang[3] = l_j+size_cell_y
                    #cv2.rectangle(img,(k_i, l_j),(k_i+dev1, l_j+dev2),(0,255,255),3)                  
                l_j+= dev2
            k_i+= dev1
            l_j = 0

        return coord_rectang



    def image_resize(self, image, width = None, height = None, inter = cv2.INTER_AREA):
        # initialize the dimensions of the image to be resized and
        # grab the image size
        dim = None
        (h, w) = image.shape[:2]

        # if both the width and height are None, then return the
        # original image
        if width is None and height is None:
            return image

        # check to see if the width is None
        if width is None:
            # calculate the ratio of the height and construct the
            # dimensions
            r = height / float(h)
            dim = (int(w * r), height)

        # otherwise, the height is None
        else:
            # calculate the ratio of the width and construct the
            # dimensions
            r = width / float(w)
            dim = (width, int(h * r))

        # resize the image

        lst = list(dim)
        lst[0] = int(lst[0])
        dim = tuple(lst)

        resized = cv2.resize(image, dim, interpolation = inter)

        # return the resized image
        return resized



    def find_movement_of_man (self, curr_frame, back_frame, connectivity = 4):
    
        #shape
        shape = curr_frame.shape

        #make a photo of curr_frame
        new_frame = cv2.imwrite('opencv1.png', curr_frame)

        #make a resize
        back_frame_resize = image_resize(back_frame , width = shape[1]/4, height = shape[0]/4)
        new_frame_resize = image_resize(new_frame , width = shape[1]/4, height = shape[0]/4)

        #filter
        filtered_back_frame = cv2.blur (back_frame, (9, 9))
        filtered_new_frame = cv2.blur (new_frame, (9, 9))

        #difference between photos
        diff_between = cv2.absdiff(filtered_back_frame,filtered_new_frame)


        img_grey = cv2.cvtColor(diff_between, cv2.COLOR_BGR2GRAY)

        gaussian_blur = cv2.GaussianBlur(img_grey, (5,5), 0)

        #binarization
        _, frame = cv2.threshold(gaussian_blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        kernel = np.ones((10,10),np.uint8)
        erosia = cv2.erode(frame,kernel,iterations = 1)

        frame_end = image_resize(erosia , width = sh[1], height = sh[0])

        #find the lagest component of connectivity
        output = cv2.connectedComponentsWithStats(d, connectivity, cv2.CV_32S)

        #rectangle around area

        new_rectangle_coord = [0, 0, 0, 0]
        for i in range(output[0]):
                if output[2][i][4] >= 5000: 
                    new_rectangle_coord[0] = output[2][i][0]
                    new_rectangle_coord[1] = output[2][i][1]
                    new_rectangle_coord[2] = output[2][i][0] + output[2][i][2]
                    new_rectangle_coord[3] = output[2][i][1] + output[2][i][3]
                    #cv2.rectangle(d, (output[2][i][0], output[2][i][1]), 
                    ####(output[2][i][0] + output[2][i][2], output[2][i][1] + output[2][i][3]), (255,0,255), 2)

        return new_rectangle_coord

    def __del__(self):
        cap.release()

In [ ]:
import numpy as np
import cv2
import time
from tkinter import *
from IPython.display import clear_output

hsv_min = np.array((53, 55, 147), np.uint8)
hsv_max = np.array((83, 160, 255), np.uint8)
colors = [hsv_min, hsv_max]

cellx = 8
celly = 8 

cum_num = 0

mark_track = marker_tracker(cam_num, cellx, celly, colors):

cv2.namedWindow ("result", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("result", (mark_track.width, mark_track.height))

while True:
    img = mark_track.img
    
    mark_track.draw_grid()
    
    marker_coord = mark_track.detect_marker(img, colors)
    
    mark_track.find_marker_place (img, marker_coord, size_cell_x, size_cell_y, color)
    
    
    if up != [0, 0] and right != [0, 0] and left != [0, 0] and down != [0, 0]:
        cv2.rectangle(img,(left[0], up[1]),(right[0]+dev1, down[1]+dev2),(255,0,255),3)
        

                     
    i = 0
    j = 0   
    while i <= img.shape[1]:#width:
        while j <= img.shape[0]:#height:
            cv2.rectangle(img,(i, j),(i+dev1, j+dev2),(255,255,255),1)
            j+= dev2
        i+= dev1
        j = 0 
    
    if dArea > 100:
        x = int(dM10 / dArea)
        y = int(dM01 / dArea)
        cv2.circle(img, (x, y), 10, (0,0,255), -1)
        
        k_i = 0
        l_j = 0
        while k_i <= img.shape[1]:
            while l_j <= img.shape[0]:
                if x < k_i + dev1 and x > k_i and y < l_j + dev2 and y > l_j:
                    cv2.rectangle(img,(k_i, l_j),(k_i+dev1, l_j+dev2),(0,255,255),3)
                    
                    if up == [0, 0] or right == [0, 0] or left == [0, 0] or down == [0, 0]:
                        x_array.append(k_i)
                        y_array.append(l_j)       
                        
                l_j+= dev2
            k_i+= dev1
            l_j = 0
    
    keyb = cv2.waitKey(1)
    
    if (keyb & 0xFF == ord ('c')):
            up, left, right, down = measurments(x_array, y_array)
                    
    if keyb%256 == 32:
        # SPACE pressed
        photo = make_photo(img)
        print(photo)
        print("written!")
        
#     timer+=1
#     print(timer)
    
    cv2.imshow('result', img)
    
    if (keyb & 0xFF == ord ('q')):
        break

cap.release()
cv2.destroyAllWindows()
    